# RAG System Comprehensive Evaluation\n\nThis notebook evaluates the IZU RAG chatbot system on 50 test questions.\n\n## Metrics Evaluated:\n- **Retrieval Quality**: URL coverage, topic coverage, relevance\n- **Answer Quality**: Semantic similarity, keyword overlap\n- **Performance**: Response time, throughput\n- **Cost**: Token usage, API costs\n\n## Setup

In [ ]:
import json\nimport numpy as np\nimport faiss\nimport openai\nfrom dotenv import load_dotenv\nimport os\nimport time\nfrom datetime import datetime\nimport pandas as pd\nfrom tqdm import tqdm\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.metrics.pairwise import cosine_similarity\n\n# Setup\nload_dotenv()\nopenai.api_key = os.getenv('OPENAI_API_KEY')\n\nprint(\"✓ Imports loaded\")

In [ ]:
print(\"Loading RAG system resources...\")\n\n# Load chunks\nwith open('chunks.json', 'r', encoding='utf-8') as f:\n    chunks = json.load(f)\nprint(f\"✓ Loaded {len(chunks)} chunks\")\n\n# Load FAISS index\nindex = faiss.read_index('faiss_index.bin')\nprint(f\"✓ Loaded FAISS index: {index.ntotal} vectors\")\n\n# Load F test dataset\nwith open('test_dataset.json', 'r', encoding='utf-8') as f:\n    test_questions = json.load(f)\nprint(f\"✓ Loaded {len(test_questions)} test questions\")\n\nprint(f\"\\n{'='*60}\")\nprint(\"SYSTEM READY FOR EVALUATION\")\nprint(f\"{'='*60}\")

In [ ]:
def get_embedding(text):\n    \"\"\"Get embedding for text\"\"\"\n    response = openai.embeddings.create(\n        input=[text.replace(\"\\n\", \" \")],\n        model=\"text-embedding-3-small\"\n    )\n    return response.data[0].embedding\n\ndef retrieve_chunks(query, top_k=5):\n    \"\"\"Retrieve most relevant chunks\"\"\"\n    query_embedding = np.array([get_embedding(query)], dtype='float32')\n    faiss.normalize_L2(query_embedding)\n    scores, indices = index.search(query_embedding, top_k)\n    \n    results = []\n    for idx, score in zip(indices[0], scores[0]):\n        results.append({\n            'content': chunks[idx]['content'],\n            'metadata': chunks[idx]['metadata'],\n            'score': float(score)\n        })\n    return results\n\ndef answer_question(query, top_k=5):\n    \"\"\"Answer question using RAG\"\"\"\n    start_time = time.time()\n    \n    # Retrieve\n    retrieval_start = time.time()\n    retrieved = retrieve_chunks(query, top_k)\n    retrieval_time = time.time() - retrieval_start\n    \n    # Build context\n    context = \"\\n---\\n\".join([\n        f\"Kaynak: {c['metadata']['title']}\\n{c['content']}\"\n        for c in retrieved\n    ])\n    \n    # Generate\n    generation_start = time.time()\n    response = openai.chat.completions.create(\n        model=\"gpt-4o-mini\",\n        messages=[\n            {\"role\": \"system\", \"content\": \"Sen İZÜ için bir asistansın. Sadece verilen bilgileri kullan. Türkçe cevap ver.\"},\n            {\"role\": \"user\", \"content\": f\"Context:\\n{context}\\n\\nSoru: {query}\"}\n        ],\n        temperature=0.3,\n        max_tokens=500\n    )\n    generation_time = time.time() - generation_start\n    total_time = time.time() - start_time\n    \n    return {\n        'answer': response.choices[0].message.content,\n        'retrieved_chunks': retrieved,\n        'retrieval_time': retrieval_time,\n        'generation_time': generation_time,\n        'total_time': total_time,\n        'tokens_used': response.usage.total_tokens,\n        'prompt_tokens': response.usage.prompt_tokens,\n        'completion_tokens': response.usage.completion_tokens\n    }\n\nprint(\"✓ RAG functions defined\")

In [ ]:
def calculate_semantic_similarity(text1, text2):\n    \"\"\"Calculate semantic similarity between two texts\"\"\"\n    emb1 = np.array(get_embedding(text1))\n    emb2 = np.array(get_embedding(text2))\n    return float(cosine_similarity([emb1], [emb2])[0][0])\n\ndef calculate_keyword_overlap(text1, text2):\n    \"\"\"Calculate keyword overlap between texts\"\"\"\n    words1 = set(text1.lower().split())\n    words2 = set(text2.lower().split())\n    if not words1 or not words2:\n        return 0.0\n    intersection = words1.intersection(words2)\n    union = words1.union(words2)\n    return len(intersection) / len(union)\n\ndef evaluate_retrieval(retrieved_chunks, expected_topics, expected_urls):\n    \"\"\"Evaluate retrieval quality\"\"\"\n    # URL coverage\n    retrieved_urls = [c['metadata']['url'] for c in retrieved_chunks]\n    url_matches = sum(1 for url in expected_urls if any(exp in url for exp in retrieved_urls))\n    url_coverage = url_matches / len(expected_urls) if expected_urls else 0\n    \n    # Topic coverage\n    all_content = ' '.join([c['content'].lower() for c in retrieved_chunks])\n    topic_matches = sum(1 for topic in expected_topics if topic.lower() in all_content)\n    topic_coverage = topic_matches / len(expected_topics) if expected_topics else 0\n    \n    # Average relevance score\n    avg_score = np.mean([c['score'] for c in retrieved_chunks]) if retrieved_chunks else 0\n    \n    return {\n        'url_coverage': url_coverage,\n        'topic_coverage': topic_coverage,\n        'avg_relevance_score': float(avg_score)\n    }\n\nprint(\"✓ Evaluation functions defined\")

In [ ]:
print(\"="*80)\nprint(\"RUNNING COMPREHENSIVE EVALUATION\")\nprint(\"="*80)\nprint()\n\nresults = []\n\nfor i, question_data in enumerate(tqdm(test_questions, desc=\"Evaluating\"), 1):\n    try:\n        # Get question\n        question = question_data['question_tr']\n        \n        # Get answer\n        result = answer_question(question)\n        \n        # Evaluate retrieval\n        retrieval_metrics = evaluate_retrieval(\n            result['retrieved_chunks'],\n            question_data.get('expected_topics', []),\n            question_data.get('requires_urls', [])\n        )\n        \n        # Evaluate answer quality\n        semantic_sim = calculate_semantic_similarity(\n            result['answer'],\n            question_data.get('ground_truth_answer', '')\n        )\n        \n        keyword_overlap = calculate_keyword_overlap(\n            result['answer'],\n            question_data.get('ground_truth_answer', '')\n        )\n        \n        # Calculate cost (GPT-4o-mini pricing)\n        cost = (result['prompt_tokens'] * 0.15 / 1_000_000) + \\\n               (result['completion_tokens'] * 0.6 / 1_000_000)\n        \n        # Store results\n        results.append({\n            'question_id': question_data['id'],\n            'question': question,\n            'category': question_data['category'],\n            'difficulty': question_data['difficulty'],\n            'answer': result['answer'],\n            'ground_truth': question_data.get('ground_truth_answer', ''),\n            'semantic_similarity': semantic_sim,\n            'keyword_overlap': keyword_overlap,\n            'url_coverage': retrieval_metrics['url_coverage'],\n            'topic_coverage': retrieval_metrics['topic_coverage'],\n            'avg_relevance_score': retrieval_metrics['avg_relevance_score'],\n            'total_time_ms': result['total_time'] * 1000,\n            'retrieval_time_ms': result['retrieval_time'] * 1000,\n            'generation_time_ms': result['generation_time'] * 1000,\n            'tokens_used': result['tokens_used'],\n            'cost_usd': cost\n        })\n        \n    except Exception as e:\n        print(f\"\\nError on question {i}: {e}\")\n        continue\n\n# Convert to DataFrame\ndf = pd.DataFrame(results)\n\nprint(f\"\\n✓ Evaluation complete!\")\nprint(f\"✓ Evaluated {len(results)} questions\")

In [ ]:
print(\"="*80)\nprint(\"OVERALL METRICS\")\nprint(\"="*80)\nprint()\n\nprint(\"Answer Quality:\")\nprint(f\"  Semantic Similarity: {df['semantic_similarity'].mean():.3f}\")\nprint(f\"  Keyword Overlap: {df['keyword_overlap'].mean():.3f}\")\nprint()\n\nprint(\"Retrieval Quality:\")\nprint(f\"  URL Coverage: {df['url_coverage'].mean():.1%}\")\nprint(f\"  Topic Coverage: {df['topic_coverage'].mean():.1%}\")\nprint(f\"  Avg Relevance Score: {df['avg_relevance_score'].mean():.3f}\")\nprint()\n\nprint(\"Performance:\")\nprint(f\"  Avg Total Time: {df['total_time_ms'].mean():.0f}ms\")\nprint(f\"  Avg Retrieval Time: {df['retrieval_time_ms'].mean():.0f}ms\")\nprint(f\"  Avg Generation Time: {df['generation_time_ms'].mean():.0f}ms\")\nprint(f\"  95th Percentile: {df['total_time_ms'].quantile(0.95):.0f}ms\")\nprint()\n\nprint(\"Cost:\")\nprint(f\"  Avg Cost per Question: ${df['cost_usd'].mean():.4f}\")\nprint(f\"  Total Cost: ${df['cost_usd'].sum():.2f}\")\nprint(f\"  Avg Tokens: {df['tokens_used'].mean():.0f}\")\nprint()\n\nprint(\"Projected Costs:\")\nprint(f\"  1,000 questions: ${df['cost_usd'].mean() * 1000:.2f}\")\nprint(f\"  10,000 questions: ${df['cost_usd'].mean() * 10000:.2f}\")\nprint(f\"  100,000 questions: ${df['cost_usd'].mean() * 100000:.2f}\")

In [ ]:
print(\"="*80)\nprint(\"PERFORMANCE BY CATEGORY\")\nprint(\"="*80)\nprint()\n\ncategory_stats = df.groupby('category').agg({\n    'semantic_similarity': 'mean',\n    'keyword_overlap': 'mean',\n    'total_time_ms': 'mean',\n    'cost_usd': 'mean'\n}).round(3)\n\nprint(category_stats.to_string())\nprint()\n\n# Best and worst categories\nbest_cat = category_stats['semantic_similarity'].idxmax()\nworst_cat = category_stats['semantic_similarity'].idxmin()\n\nprint(f\"Best Category: {best_cat} ({category_stats.loc[best_cat, 'semantic_similarity']:.3f})\")\nprint(f\"Worst Category: {worst_cat} ({category_stats.loc[worst_cat, 'semantic_similarity']:.3f})\")

In [ ]:
# Setup plotting\nplt.style.use('seaborn-v0_8-darkgrid')\nfig, axes = plt.subplots(2, 2, figsize=(15, 10))\n\n# 1. Semantic Similarity by Category\ndf.groupby('category')['semantic_similarity'].mean().sort_values().plot(kind='barh', ax=axes[0,0], color='skyblue')\naxes[0,0].set_title('Semantic Similarity by Category')\naxes[0,0].set_xlabel('Similarity Score')\naxes[0,0].axvline(0.6, color='green', linestyle='--', label='Good (>0.6)')\naxes[0,0].legend()\n\n# 2. Response Time Distribution\naxes[0,1].hist(df['total_time_ms'], bins=20, color='lightcoral', edgecolor='black')\naxes[0,1].set_title('Response Time Distribution')\naxes[0,1].set_xlabel('Time (ms)')\naxes[0,1].set_ylabel('Frequency')\naxes[0,1].axvline(3000, color='green', linestyle='--', label='Target (<3s)')\naxes[0,1].legend()\n\n# 3. Performance by Difficulty\ndf.groupby('difficulty')['semantic_similarity'].mean().plot(kind='bar', ax=axes[1,0], color='lightgreen')\naxes[1,0].set_title('Performance by Difficulty')\naxes[1,0].set_ylabel('Semantic Similarity')\naxes[1,0].set_xticklabels(axes[1,0].get_xticklabels(), rotation=0)\n\n# 4. Cost vs Performance\naxes[1,1].scatter(df['cost_usd'] * 1000, df['semantic_similarity'], alpha=0.6, color='purple')\naxes[1,1].set_title('Cost vs Performance')\naxes[1,1].set_xlabel('Cost (cents)')\naxes[1,1].set_ylabel('Semantic Similarity')\naxes[1,1].grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.savefig('evaluation_dashboard.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(\"✓ Dashboard saved: evaluation_dashboard.png\")

In [ ]:
print(\"="*80)\nprint(\"SAMPLE RESULTS\")\nprint(\"="*80)\nprint()\n\n# Show best and worst examples\nbest_idx = df['semantic_similarity'].idxmax()\nworst_idx = df['semantic_similarity'].idxmin()\n\nprint(\"BEST RESULT:\")\nprint(f\"Question: {df.loc[best_idx, 'question']}\")\nprint(f\"Similarity: {df.loc[best_idx, 'semantic_similarity']:.3f}\")\nprint(f\"Answer: {df.loc[best_idx, 'answer'][:200]}...\")\nprint()\n\nprint(\"WORST RESULT:\")\nprint(f\"Question: {df.loc[worst_idx, 'question']}\")\nprint(f\"Similarity: {df.loc[worst_idx, 'semantic_similarity']:.3f}\")\nprint(f\"Answer: {df.loc[worst_idx, 'answer'][:200]}...\")\nprint(f\"Ground Truth: {df.loc[worst_idx, 'ground_truth'][:200]}...\")

In [ ]:
# Save detailed results\ntimestamp = datetime.now().strftime('%Y%m%d_%H%M%S')\n\n# CSV\ncsv_file = f'evaluation_results_{timestamp}.csv'\ndf.to_csv(csv_file, index=False)\nprint(f\"✓ Saved: {csv_file}\")\n\n# JSON\njson_file = f'evaluation_results_{timestamp}.json'\ndf.to_json(json_file, orient='records', indent=2)\nprint(f\"✓ Saved: {json_file}\")\n\n# Summary report\nreport = f\"\"\"\n# RAG Evaluation Report\nGenerated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n## Overall Metrics\n- Semantic Similarity: {df['semantic_similarity'].mean():.3f}\n- URL Coverage: {df['url_coverage'].mean():.1%}\n- Avg Response Time: {df['total_time_ms'].mean():.0f}ms\n- Avg Cost: ${df['cost_usd'].mean():.4f}\n\n## Performance by Category\n{category_stats.to_markdown()}\n\n## Status\n{'PASS: System meets quality targets' if df['semantic_similarity'].mean() > 0.6 else 'FAIL: System below quality targets'}\n\"\"\"\n\nreport_file = f'evaluation_report_{timestamp}.md'\nwith open(report_file, 'w') as f:\n    f.write(report)\n\nprint(f\"✓ Saved: {report_file}\")\nprint()\nprint(\"="*80)\nprint(\"EVALUATION COMPLETE\")\nprint(\"="*80)